In [14]:
import pandas as pd
from google.colab import drive

# Mount Google Drive (if not already mounted)
drive.mount('/content/drive')

# Define file paths
github_emissions_url = "https://raw.githubusercontent.com/apownukepcc/spring-2025-datathon/main/004-Emissions-Data.csv"
github_weather_url = "https://raw.githubusercontent.com/apownukepcc/spring-2025-datathon/main/001-Weather.csv"
output_file = "/content/drive/My Drive/006-Dataset-For-Predictions.csv"  # Before cleaning
cleaned_output_file = "/content/drive/My Drive/007-Dataset-For-Predictions-Cleaned.csv"  # After cleaning

# Load the emissions dataset
try:
    emissions_data = pd.read_csv(github_emissions_url)
    print("Emissions dataset successfully loaded from GitHub.")
except Exception as e:
    print(f"Error loading emissions dataset: {e}")
    exit()

# Print columns for verification
print("Emissions Data Columns:", emissions_data.columns.tolist())

# Convert 'TimeStamp' to date-only format
if "TimeStamp" not in emissions_data.columns:
    print("Error: 'TimeStamp' column not found in emissions data.")
    exit()
else:
    emissions_data["date"] = pd.to_datetime(emissions_data["TimeStamp"]).dt.date

# Pivot the emissions dataset
try:
    final_data = emissions_data.pivot_table(index=["date", "Source"],
                                            columns="Parameter",
                                            values="Value").reset_index()
    print("Emissions data successfully pivoted.")
except Exception as e:
    print(f"Error during pivoting the data: {e}")
    exit()

# Load the weather dataset
try:
    weather_data = pd.read_csv(github_weather_url)
    print("Weather dataset successfully loaded from GitHub.")

    # Convert weather dataset 'date' column to match emissions dataset
    if "date" not in weather_data.columns and "Date" in weather_data.columns:
        weather_data.rename(columns={"Date": "date"}, inplace=True)

    if "date" in weather_data.columns:
        weather_data["date"] = pd.to_datetime(weather_data["date"]).dt.date
        # Drop rows where 'date' is NaN
        weather_data = weather_data.dropna(subset=["date"])
    else:
        print("Error: 'date' column not found in weather data. Skipping merge.")
        weather_data = None
except Exception as e:
    print(f"Error loading weather dataset: {e}")
    weather_data = None

# Merge only if weather data is available
if weather_data is not None:
    try:
        final_data_with_weather = pd.merge(final_data, weather_data, on="date", how="left")
        print("Weather data successfully merged.")
        print(f"Number of rows after merging: {len(final_data_with_weather)}")
    except Exception as e:
        print(f"Error merging weather data: {e}")
else:
    final_data_with_weather = final_data  # Use emissions dataset without merging

# **Re-check column names after merging**
print("Columns in merged dataset:", final_data_with_weather.columns.tolist())

# Save dataset BEFORE cleaning
try:
    final_data_with_weather.to_csv(output_file, index=False)
    print(f"Dataset before cleaning saved to: {output_file}")
except Exception as e:
    print(f"Error saving dataset before cleaning: {e}")

# **Step 1: Remove Unwanted Columns (`wpgt`, `tsun`)**
columns_to_remove = ["wpgt", "tsun"]
existing_columns = final_data_with_weather.columns.tolist()

# Remove columns only if they exist in the dataset
for col in columns_to_remove:
    if col in existing_columns:
        final_data_with_weather.drop(columns=[col], inplace=True)
        print(f"Column '{col}' removed from dataset.")

# **Step 2: Drop Rows with Missing Values**
df_cleaned = final_data_with_weather.dropna()

# Check the number of rows remaining after cleaning
num_rows_after_cleaning = len(df_cleaned)

# Save the cleaned dataset **only if it is not empty**
if num_rows_after_cleaning > 0:
    try:
        df_cleaned.to_csv(cleaned_output_file, index=False)
        print(f"Cleaned dataset saved to: {cleaned_output_file}")
    except Exception as e:
        print(f"Error saving cleaned dataset: {e}")
else:
    print("Warning: The dataset is empty after cleaning!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Emissions dataset successfully loaded from GitHub.
Emissions Data Columns: ['Plant', 'Source', 'Parameter', 'Units', 'TimeStamp', 'Value', 'Description']
Emissions data successfully pivoted.
Weather dataset successfully loaded from GitHub.
Weather data successfully merged.
Number of rows after merging: 2269
Columns in merged dataset: ['date', 'Source', 'COTONS', 'GFLOW_BA', 'GFLOW_BT', 'HEATINBA', 'HEATINBT', 'HEAT_QA', 'LOADMWBA', 'LOADMWBT', 'NH3TONS', 'NOXTONS', 'SO2TONS', 'UNITONBA', 'UNITONBT', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
Dataset before cleaning saved to: /content/drive/My Drive/006-Dataset-For-Predictions.csv
Column 'wpgt' removed from dataset.
Column 'tsun' removed from dataset.
Cleaned dataset saved to: /content/drive/My Drive/007-Dataset-For-Predictions-Cleaned.csv
